# Homework 1

In [ ]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2018 term"

This homework covers material from the unit on distributed representations. The primary goal is to explore some new techniques for building and assessing VSMs. The code you write as part of the assignment should be useful for research involving vector representations as well.

Like all homeworks, this should be submitted via Canvas. All you have to do is paste in your answers (which are all numerical values) and include the SUNetIds of anyone you worked with. Here's a direct link to the homework form:

https://canvas.stanford.edu/courses/83399/quizzes/50268

__Contents__

0. [Questions 1–2: Dice distance [2 points]](#Questions-1–2:-Dice-distance-[2-points])
0. [Question 3: t-test reweighting [2 points]](#Question-3:-t-test-reweighting-[2-points])
0. [Questions 4–6: Reweighting and co-occurrence frequency [3 points]](#Questions-4–6:-Reweighting-and-co-occurrence-frequency-[3-points])
0. [Question 7: Meeting the GloVe objective [1 point]](#Question-7:-Meeting-the-GloVe-objective-[1-point])
0. [Question 8: Expressive eloooongation [2 points]](#Question-8:-Expressive-eloooongation-[2-points])

In [6]:
import numpy as np
import os
import pandas as pd
from mittens import GloVe
from scipy.stats import pearsonr
import vsm

## Questions 1–2: Dice distance [2 points]

First, implement [Dice distance](https://en.wikipedia.org/wiki/Sørensen–Dice_coefficient) for real-valued vectors of dimension $n$, as

$$\textbf{dice}(u, v) = 1 - \frac{
    2 \sum_{i=1}^{n}\min(u_{i}, v_{i})
}{
    \sum_{i=1}^{n} u_{i} + v_{i}
}$$

(You can use `vsm.matching` for part of this.)

Second, you might want to test your implementation. Here's a simple function for that:

In [10]:
def dice_distance(u, v):
    numerator = 2 * vsm.matching(u, v)
    denominator = np.linalg.norm(u, ord=1) + np.linalg.norm(v, ord=1)
    
    return 1 - numerator / denominator

In [8]:
def test_dice_implementation(func):
    """`func` should be an implementation of `dice` as defined above."""
    X = np.array([
        [  4.,   4.,   2.,   0.],
        [  4.,  61.,   8.,  18.],
        [  2.,   8.,  10.,   0.],
        [  0.,  18.,   0.,   5.]]) 
    assert func(X[0], X[1]).round(5) == 0.80198
    assert func(X[1], X[2]).round(5) == 0.67568

In [11]:
test_dice_implementation(dice_distance)

Third, use your implementation to measure the distance between A and B and between B and C in the toy `ABC` matrix we used in the first VSM notebook, repeated here for convenience.

In [12]:
ABC = pd.DataFrame([
    [ 2.0,  4.0], 
    [10.0, 15.0], 
    [14.0, 10.0]],
    index=['A', 'B', 'C'],
    columns=['x', 'y']) 

ABC

,x,y
A,2.0,4.0
B,10.0,15.0
C,14.0,10.0


__To submit:__

1. Dice distance between A and B.
2. Dice distance between B and C.

(The real question, which these values answer, is whether this measure place A and B close together relative to B and C – our goal for that example.) 

In [18]:
dice_distance(ABC.loc['A',:], ABC.loc['B',:])

0.6129032258064516

In [19]:
dice_distance(ABC.loc['B',:], ABC.loc['C',:])

0.18367346938775508

## Question 3: t-test reweighting [2 points]

The t-test statistic can be thought of as a reweighting scheme. For a count matrix $X$, row index $i$, and column index $j$:

$$\textbf{ttest}(X, i, j) = 
\frac{
    P(X, i, j) - \big(P(X, i, *)P(X, *, j)\big)
}{
\sqrt{(P(X, i, *)P(X, *, j))}
}$$

where $P(X, i, j)$ is $X_{ij}$ divided by the total values in $X$, $P(X, i, *)$ is the sum of the values in row $i$ of $X$ divided by the total values in $X$, and $P(X, *, j)$ is the sum of the values in column $j$ of $X$ divided by the total values in $X$.

First, implement this reweighting scheme.

In [34]:
def reweight_ttest(X_df):
    X = X_df.values
    reweighted_X = X.copy()
    
    full_sum = np.sum(X)
    row_sums = [np.sum(X[i,:]) for i in range(X.shape[0])]
    column_sums = [np.sum(X[:,j]) for j in range(X.shape[1])]
    
    row_sums_normalized = row_sums / full_sum
    column_sums_normalized = column_sums / full_sum
    
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            row_column_norm_product = row_sums_normalized[i] * column_sums_normalized[j]
            reweighted_X[i,j] = (X[i,j] / full_sum - row_column_norm_product) / np.sqrt(row_column_norm_product)
            
    return reweighted_X
    

X = pd.DataFrame(np.array([
        [  4.,   4.,   2.,   0.],
        [  4.,  61.,   8.,  18.],
        [  2.,   10.,  10.,   0.],
        [  0.,  18.,   0.,   5.]]))
reweight_ttest(X)

array([[ 0.33150685, -0.07771087,  0.04455741, -0.10387501],
       [-0.07401914,  0.03298289, -0.10467887,  0.08009701],
       [ 0.03324821, -0.08873439,  0.33305885, -0.15407153],
       [-0.10387501,  0.07241868, -0.14690144,  0.05985706]])

Second, test your implementation:

In [38]:
def test_ttest_implementation(func):
    """`func` should be an implementation of ttest reweighting as defined above."""
    X = pd.DataFrame(np.array([
        [  4.,   4.,   2.,   0.],
        [  4.,  61.,   8.,  18.],
        [  2.,   8.,  10.,   0.],
        [  0.,  18.,   0.,   5.]]))    
    actual = np.array([
        [ 0.33056, -0.07689,  0.04321, -0.10532],
        [-0.07689,  0.03839, -0.10874,  0.07574],
        [ 0.04321, -0.10874,  0.36111, -0.14894],
        [-0.10532,  0.07574, -0.14894,  0.05767]])    
    predicted = func(X)
    assert np.array_equal(predicted.round(5), actual)
    
test_ttest_implementation(reweight_ttest)

Third, apply your implementation to the matrix stored in `imdb_window5-scaled.csv.gz`.

__To submit__: the cell value for the row labeled _superb_ and the column labeled _movie_.

(The goal here is really to obtain a working implementation of $\textbf{ttest}$. It could be an ingredient in a winning bake-off entry!)

In [41]:
imdb5 = pd.read_csv('vsmdata/imdb_window5-scaled.csv.gz', index_col=0)

In [43]:
reweighted_imdb5 = reweight_ttest(imdb5)

In [57]:
reweighted_imdb5[imdb5.index.get_loc('superb'), imdb5.index.get_loc('movie')]

-0.0008427614547882469

## Questions 4–6: Reweighting and co-occurrence frequency [3 points]

We've seen that raw count matrices encode a lot of frequency information. This is not necessarily all bad (stronger words like _superb_ will be rarer than weak ones like _good_ in part because of their more specialized semantics), but we do hope that our reweighting schemes will get us away from these relatively mundane associations. Thus, for any reweighting scheme, we should ask about its correlation with the raw co-occurrence counts.

Your task: using [scipy.stats.pearsonr](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), calculate the Pearson correlation coefficient between the raw count values of `imdb5` as loaded in the previous question and the values obtained from applying PMI and Positive PMI to this matrix, and from reweighting each row by its length norm (as defined in the first noteboook for this unit; `vsm.length_norm`). Note: `X.values.ravel()` will give you the vector of values in the `pd.DataFrame` instance `X`.

__To submit:__

1. Correlation coefficient for the PMI comparison.
1. Correlation coefficient for the Positive PMI comparison.
1. Correlation coefficient for the length-norm comparison.

(The hope is that seeing these values will give you a better sense for how these reweighting schemes compare to the input count matrices.)

In [60]:
pmi_normalized_imdb5 = vsm.pmi(imdb5, positive=False)
ppmi_normalized_imdb5 = vsm.pmi(imdb5, positive=True)

In [71]:
length_normalized_imdb5 = imdb5.copy()
for i in range(imdb5.shape[0]):
    length_normalized_imdb5.iloc[i,:] = vsm.length_norm(length_normalized_imdb5.iloc[i,:])

In [73]:
print(pearsonr(pmi_normalized_imdb5.values.ravel(), imdb5.values.ravel()))
print(pearsonr(ppmi_normalized_imdb5.values.ravel(), imdb5.values.ravel()))
print(pearsonr(length_normalized_imdb5.values.ravel(), imdb5.values.ravel()))

(0.007653071529112714, 2.2613e-320)
(0.04498063308409067, 0.0)
(0.1221866233067643, 0.0)


## Question 7: Meeting the GloVe objective [1 point]

We saw that GloVe can be thought of as seeking vectors whose dot products are proportional to their PMI values. How close does GloVe come to this in practice? This question asks you to conduct a simple empirical assessment of that: 

1. Load the matrix stored as `imdb_window5-scaled.csv.gz` in the data distribution. Call this `imdb5`.
2. Reweight `imdb5` with Positive PMI.
3. Run GloVe on `imdb5` for 10 iterations, learning vectors of dimension 20 (`n=20`). Definitely use the implementation in the `mittens` package, not in `vsm.glove`, else this will take way too long. Except for `max_iter` and `n`, use all the default parameters.
4. Report the correlation between the cell values in the PMI and GloVe versions. For this, you can include all 0 values (even though GloVe ignores them). Use `pearsonr` as above.

In [92]:
glove = GloVe(n=20, max_iter=100)
glove_vectors = glove.fit(imdb5.values)

Iteration 100: error 325243.0612

In [95]:
pmi_approximation = imdb5.values.copy()
for i in range(imdb5.shape[0]):
    for j in range(imdb5.shape[1]):
        pmi_approximation[i,j] = np.dot(glove_vectors[i], glove_vectors[j])

In [97]:
pearsonr(ppmi_normalized_imdb5.values.ravel(), pmi_approximation.ravel())

(0.027687433235751093, 0.0)

In [94]:
imdb5_glv = pd.DataFrame(glove_vectors, index=imdb5.index)
vsm.neighbors('superb', imdb5_glv).head()

superb         0.000000
strong         0.008243
terrific       0.009676
direction      0.011180
outstanding    0.012509
dtype: float64

## Question 8: Expressive eloooongation [2 points]

One of the goals of subword modeling is to capture out-of-vocabulary (OOV) words. This is particularly important for __expressive elogations__ like _coooooool_ and _booriiiing_. Because the amount of elongation is highly variable, we're unlikely to have good representations for such words. How does [our simple approach to subword modeling](vsm_01_distributional.ipynb#Subword-information) do with these phenomena?

__Your task:__

* Use `vsm.ngram_vsm` to create a 4-gram character-level VSM from the matrix in `imdb_window20-flat.csv.gz`.

* Using `character_level_rep` from the notebook for representing words in this space, calculate the cosine distances for pair `cool` and `cooooool`.

__To submit__: the cosine distance  between `cool` and `cooooool`

(Of course, the broader question we want to answer is whether these words are being modeled as similar, which is a more subjective, comparative question. It does depend on these distance calculations, though.)

In [98]:
imdb20 = pd.read_csv('vsmdata/imdb_window20-flat.csv.gz', index_col=0)

In [111]:
four_gram_vectors = vsm.ngram_vsm(imdb20, n=4)

In [113]:
def character_level_rep(word, cf, n=4):
    ngrams = vsm.get_character_ngrams(word, n)
    ngrams = [n for n in ngrams if n in cf.index]  
    reps = cf.loc[ngrams].values
    
    return reps.sum(axis=0)

In [114]:
cool_rep = character_level_rep('cool', four_gram_vectors, n=4)
cooooool_rep = character_level_rep('cooooool', four_gram_vectors, n=4)

vsm.cosine(cool_rep, cooooool_rep)

0.0006569654844812423